# 2022 / 12 / 25

In [1]:
using IntervalArithmetic
using LaTeXStrings

In [2]:
include("./concatImportanceMethod/concatImportanceMethod.jl")
include("./concatMatricesMethod/concatMatricesMethod.jl")
include("./display/concatImportanceMethod.jl")
include("./display/concatMatricesMethod.jl")
include("./display/interval.jl")
include("./display/twofoldInterval.jl")
include("./twofoldIntervalPCM/twofoldIntervalPCM.jl")

intervalPCM2TwofoldIntervalPCM

In [9]:
function concatMatricesMethod(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    A = intervalPCM2TwofoldIntervalPCM(A₁, A₂)
    display(L"\text{修正前の行列}")
    display(L"A = %$(twofoldIntervalMatrixLaTeXString(A))")

    result = solveConcatMatricesMethodLP(A)
    display(L"\text{LPの最適解}")
    displayConcatMatricesMethodLPResult(result)

    Â = updateConcatMatricesMethodPCM(A, result)
    display(L"\text{修正後の行列}")
    display(L"\hat{A} = %$(twofoldIntervalMatrixLaTeXString(Â))")
end

concatMatricesMethod (generic function with 1 method)

In [10]:
function concatImportanceMethod(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    approximationResults = solveConcatImportanceMethodApproximationLP.([A₁, A₂])
    display(L"\text{各DMの上近似と下近似}")
    displayConcatImportanceMethodApproximationLPResults(approximationResults)

    display(L"\frac{w_{23}^{\text{U}+}}{w_{12}^{\text{U}-}} = %$(approximationResults[2].wₖᵁ⁺[3] / approximationResults[1].wₖᵁ⁻[2])")

    tBoundaries = calculateConcatImportanceMethodTBoundaries.(approximationResults)
    display(L"\text{各DMの上近似と下近似の}t\text{の範囲}")
    displayConcatImportanceMethodTBoundaries(tBoundaries)

    result = solveConcatImportanceMethodConcatLP(approximationResults, tBoundaries)
    display(L"\text{LPの最適解}")
    displayConcatImportanceMethodConcatLPResult(result)

    Â = generateConcatImportanceMethodPCM(result)
    display(L"\text{結合後のPCM}")
    display(L"\hat{A} = %$(twofoldIntervalMatrixLaTeXString(Â))")
end

concatImportanceMethod (generic function with 1 method)

In [11]:
function runBothMethod(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    display(L"A_1 = %$(intervalMatrixLaTeXString(A₁))")
    display(L"A_2 = %$(intervalMatrixLaTeXString(A₂))")
    
    display(L"\text{--- PCMを結合 ---}")
    concatMatricesMethod(A₁, A₂)

    display(L"\text{--- 重要度を結合 ---}")
    concatImportanceMethod(A₁, A₂)
end

runBothMethod (generic function with 1 method)

重要度を結合する方のみキャンセルが発生する

In [55]:
A₁ = [
    1..1 4..4 2..2
    (1/4)..(1/4) 1..1 (1/2)..(1/2)
    (1/2)..(1/2) 2..2 1..1
]
A₂ = deepcopy(A₁)
m,n = size(A₂)
for i = 1:n, j = 1:n
    if i == j continue end
    A₂[i,j] = (A₂[i,j].lo / 1.5)..(A₂[i,j].hi * 1.5)
end

display(L"\frac{a_{112}^\text{L}}{a_{113}^\text{L}} = %$(A₁[1,2].lo / A₁[1,3].lo)")
display(L"\frac{a_{212}^\text{L}}{a_{213}^\text{L}} = %$(A₂[1,2].lo / A₂[1,3].lo)")
display(L"\frac{a_{112}^\text{L}}{a_{213}^\text{L}} = %$(A₁[1,2].lo / A₂[1,3].lo)")
display(L"\frac{a_{212}^\text{L}}{a_{113}^\text{L}} = %$(A₂[1,2].lo / A₁[1,3].lo)")

runBothMethod(A₁, A₂)

L"$\frac{a_{112}^\text{L}}{a_{113}^\text{L}} = 2.0$"

L"$\frac{a_{212}^\text{L}}{a_{213}^\text{L}} = 2.0$"

L"$\frac{a_{112}^\text{L}}{a_{213}^\text{L}} = 3.0$"

L"$\frac{a_{212}^\text{L}}{a_{113}^\text{L}} = 1.3333333333333333$"

L"$A_1 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 4.0, 4.0 \right] & \left[ 2.0, 2.0 \right] \\ \left[ 0.25, 0.25 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.5, 0.5 \right] \\ \left[ 0.5, 0.5 \right] & \left[ 2.0, 2.0 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

...

L"$W = \begin{bmatrix}\left[ 0.571, 0.606 \right] \\ \left[ 0.143, 0.169 \right] \\ \left[ 0.225, 0.286 \right]\end{bmatrix}$"

L"$v^{\text{L}-} = \begin{bmatrix} 0.571  \\  0.143  \\  0.225 \end{bmatrix}, ~~
v^{\text{U}-} = \begin{bmatrix} 0.606  \\  0.169  \\  0.286 \end{bmatrix}, ~~
v^{\text{L}+} = \begin{bmatrix} 0.571  \\  0.143  \\  0.286 \end{bmatrix}, ~~
v^{\text{U}+} = \begin{bmatrix} 0.571  \\  0.143  \\  0.286 \end{bmatrix}
$"

L"$\varepsilon^\text{L} = \begin{bmatrix} 0.0  \\  0.0  \\  0.06 \end{bmatrix}, ~~
\varepsilon^\text{U} = \begin{bmatrix} 0.034  \\  0.026  \\  0.0 \end{bmatrix}
$"

L"$\text{結合後のPCM}$"

"\$\\hat{A} = \\begin{bmatrix} \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] & \\left[  3.38, \\left[   4.0,   4.0 \\right], 4.239 \\right] & \\left[   2.0, \\left[   2.0, 2.119 \\right], 2.686 \\right] \\\\ \\left[ 0.236, \\left[  0.25,  0.25 \\right], 0.296 \\right] & \\left[ " ⋯ 30 bytes ⋯ "ght],   1.0 \\right] & \\left[   0.5, \\left[   0.5, 0.592 \\right],  0.75 \\right] \\\\ \\left[ 0.372, \\left[ 0.472,   0.5 \\right],   0.5 \\right] & \\left[ 1.333, \\left[  1.69,   2.0 \\right],   2.0 \\right] & \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] \\end{bmatrix}\$"